# TTS Inference

This notebook can be used to generate audio samples using either NeMo's pretrained models or after training NeMo TTS models. This script currently uses a two step inference procedure. First, a model is used to generate a mel spectrogram from text. Second, a model is used to generate audio from a mel spectrogram.

Currently supported models are:
Mel Spectrogram Generators:
- Tacotron 2
- Glow-TTS

Audio Generators
- Grifflin-Lim
- WaveGlow

# Licence

> Copyright 2020 NVIDIA. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, uncomment and run this cell.
# !pip install wget
# !apt-get install sox libsndfile1 ffmpeg
# !pip install unidecode
# !pip install nemo_toolkit[tts]

# !mkdir configs
# !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/candidate/examples/tts/conf/tacotron2.yaml
# !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/candidate/examples/tts/conf/waveglow.yaml
# !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/candidate/examples/tts/conf/glow_tts.yaml
# CONFIG_PATH = "conf/"

In [ ]:
from pathlib import Path

try: CONFIG_PATH
except NameError: CONFIG_PATH = Path("../../examples/tts") / "conf"

supported_spec_gen = ["tacotron2", "glow_tts"]
supported_audio_gen = ["griffin_lim", "waveglow"]
supported_audio_gen_req_checkpoint = ["waveglow"]

print("Choose one of the following spectrogram generators:")
print([model for model in supported_spec_gen])
spectrogram_generator = input()
print("Choose one of the following audio generators:")
print([model for model in supported_audio_gen])
audio_generator = input()

assert spectrogram_generator in supported_spec_gen
assert audio_generator in supported_audio_gen

# Download pre-trained checkpoints

TODO: Enable downloading pretrained models

In [ ]:
spectrogram_generator_checkpoint_path = input(f"Input the path to the {spectrogram_generator} checkpoint: ")
audio_generator_checkpoint_path = None
if audio_generator in supported_audio_gen_req_checkpoint:
    audio_generator_checkpoint_path = input(f"Input the path to the {audio_generator} checkpoint: ")   

# Load model checkpoints

Note: For best quality with Glow TTS, please update the glow tts yaml file with the path to cmudict

In [ ]:
from omegaconf import OmegaConf, open_dict
import torch
from ruamel.yaml import YAML
from nemo.collections.asr.parts import parsers

SAMPLE_RATE = 22050
NFFT = 1024
NMEL = 80
FMAX = None

def load_spectrogram_model():
    if spectrogram_generator == "tacotron2":
        from nemo.collections.tts.models import Tacotron2Model as SpecModel
        cfg_file = Path(CONFIG_PATH) / "tacotron2.yaml"
    elif spectrogram_generator == "glow_tts":
        from nemo.collections.tts.models import GlowTTSModel as SpecModel
        cfg_file = Path(CONFIG_PATH) / "glow_tts.yaml"
    else:
        raise NotImplementedError

    with open(cfg_file) as file:
        cfg = OmegaConf.load(file)
    del cfg.model["train_ds"]
    del cfg.model["validation_ds"]
    del cfg.model["optim"]
    with open_dict(cfg):
        global SAMPLE_RATE
        global NFFT
        global NMEL
        global FMAX
        SAMPLE_RATE = cfg.sample_rate or SAMPLE_RATE
        NFFT = cfg.n_fft or NFFT
        NMEL = cfg.n_mels or NMEL
        FMAX = cfg.fmax or FMAX
    return SpecModel(cfg=cfg.model)

def load_vocoder_model():
    if audio_generator == "waveglow":
        from nemo.collections.tts.models import WaveGlowModel as VocoderModel
        cfg_file = Path(CONFIG_PATH) / "waveglow.yaml"
    elif audio_generator == "griffin_lim":
        from nemo.collections.tts.helpers.helpers import griffin_lim
        import numpy as np
        import librosa
        class GL:
            def __init__(self):
                pass
            def convert_spectrogram_to_audio(self, spec):
                log_mel_spec = spec.squeeze().to('cpu').numpy().T
                mel_spec = np.exp(log_mel_spec)
                mel_pseudo_inverse = librosa.filters.mel(SAMPLE_RATE, NFFT, NMEL, fmax=FMAX)
                return griffin_lim(np.dot(mel_spec, mel_pseudo_inverse).T ** 1.2)
            def load_state_dict(self, *args, **kwargs):
                pass
            def cuda(self, *args, **kwargs):
                return self
        return GL()
    else:
        raise NotImplementedError

    with open(cfg_file) as file:
        cfg = OmegaConf.load(file)
    del cfg.model["train_ds"]
    del cfg.model["validation_ds"]
    del cfg.model["optim"]
    with open_dict(cfg):
        if cfg.sample_rate is not None:
            assert vocoder_sample_rate == SAMPLE_RATE
        if cfg.n_fft is not None:
            assert vocoder_sample_rate == NFFT
        if cfg.n_mels is not None:
            assert vocoder_sample_rate == NMEL
        if cfg.fmax is not None:
            assert vocoder_sample_rate == FMAX
    return VocoderModel(cfg=cfg.model)

spec_gen = load_spectrogram_model()
vocoder = load_vocoder_model()

spec_gen.load_state_dict(torch.load(spectrogram_generator_checkpoint_path)["state_dict"])
if audio_generator_checkpoint_path is not None:
    vocoder.load_state_dict(torch.load(audio_generator_checkpoint_path)["state_dict"])

spec_gen = spec_gen.cuda()
vocoder = vocoder.cuda()

In [ ]:
def infer(spec_gen_model, vocder_model, str_input):
    with torch.no_grad():
        parsed = spec_gen.parse(text_to_generate)
        spectrogram = spec_gen.generate_spectrogram(tokens=parsed)
        audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
    if isinstance(spectrogram, torch.Tensor):
        spectrogram = spectrogram.to('cpu').numpy()
    if len(spectrogram.shape) == 3:
        spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

In [ ]:
text_to_generate = input("Input what you want the model to say: ")
spec, audio = infer(spec_gen, vocoder, text_to_generate)

# Show Audio and Spectrogram

In [ ]:
import IPython.display as ipd
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt


ipd.Audio(audio, rate=SAMPLE_RATE)

In [ ]:
%matplotlib inline
imshow(spec, origin="lower")
plt.show()